For the tabular portion of the multimodal probabilistic model fusion, Gaussian Process is used, as it is particularly effective in capturing trends in time-series data, even with a small and irregular dataset. Gaussian Process is a stochastic learning method that learns the transition probability in a continuous time-scale by conditioning each discrete time-series data point with interpolated time intervals.

In [17]:
import numpy as np
import pandas as pd
from pathlib import Path
import torch
import gpytorch
from gpytorch.kernels import ScaleKernel, MaternKernel, RBFKernel, IndexKernel
from gpytorch.means import ConstantMean
from gpytorch.likelihoods import GaussianLikelihood
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

For train-test split, it is done by unique patient IDs, as each row in the CSV is not a unique entry, but a series of rows for each patient ID is. Thus, the corresponding y (outcome) only is the FVC value in the last recorded week for each patient. 

Then, numerical features are normalized to ensure their equal weightage, followed by conversion to pyTorch tensors.



In [18]:
BASE_DIR = Path().resolve()
csv_path = BASE_DIR.parent / "data" / "train.csv"

df = pd.read_csv(csv_path).drop(columns = ["Percent"])

ids = df['Patient'].unique()
train_ids, val_ids = train_test_split(ids, test_size=0.2, random_state=3244)

train_df = df[df['Patient'].isin(train_ids)].reset_index(drop=True)
val_df   = df[df['Patient'].isin(val_ids)].reset_index(drop=True)

time_scaler = StandardScaler()
fvc_scaler = StandardScaler()
train_df["Weeks_scaled"] = time_scaler.fit_transform(train_df[["Weeks"]])
train_df["FVC_scaled"] = fvc_scaler.fit_transform(train_df[["FVC"]])
val_df["Weeks_scaled"] = time_scaler.transform(val_df[["Weeks"]])
val_df["FVC_scaled"] = fvc_scaler.transform(val_df[["FVC"]])
train_df = train_df.sort_values(["Patient", "Weeks"]).reset_index(drop=True)

#tidying validation dataframe (collapsing rows)
first_week = val_df.loc[val_df.groupby("Patient")["Weeks"].idxmin(), ["Patient", "Age", "Sex", "SmokingStatus", "Weeks_scaled", "FVC_scaled"]]
last_week  = val_df.loc[val_df.groupby("Patient")["Weeks"].idxmax(), ["Patient", "Weeks_scaled"]]
first_week = first_week.sort_values("Patient").reset_index(drop=True)
last_week  = last_week.sort_values("Patient").reset_index(drop=True)
dweek = last_week["Weeks_scaled"].values - first_week["Weeks_scaled"].values

val_df = val_df.loc[val_df.groupby("Patient")["Weeks_scaled"].idxmin(), ["Patient", "Age", "Sex", "SmokingStatus", "Weeks_scaled", "FVC_scaled"]]
val_df["Weeks_scaled"] = dweek
val_df = val_df.sort_values(["Patient"]).reset_index(drop=True)


Now that the preprocessing is complete, model is built with separate kernels for the time series, and each categorical feature.

In [35]:
X_time_train = torch.tensor(train_df[["Weeks_scaled"]].values, dtype=torch.float32)
X_age_train = torch.tensor(train_df[["Age"]].values, dtype=torch.float32)

sex_map = {k:i for i,k in enumerate(train_df["Sex"].astype("category").cat.categories)}
smk_map = {k:i for i,k in enumerate(train_df["SmokingStatus"].astype("category").cat.categories)}
sex_train = torch.tensor(train_df["Sex"].map(sex_map).values, dtype=torch.long).unsqueeze(-1)
smk_train = torch.tensor(train_df["SmokingStatus"].map(smk_map).values, dtype=torch.long).unsqueeze(-1)

y_train = torch.tensor(train_df["FVC_scaled"].values, dtype=torch.float32)

class MixedKernel(gpytorch.kernels.Kernel):
    has_lengthscale = False
    def __init__(self):
        super().__init__()
        self.time = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=1.5))
        self.age  = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

        self.sex  = gpytorch.kernels.IndexKernel(num_tasks=len(sex_map), rank=2)
        self.smk  = gpytorch.kernels.IndexKernel(num_tasks=len(smk_map), rank=2)

        self.register_parameter(name="raw_bias", parameter=torch.nn.Parameter(torch.tensor(0.0)))

    @property
    def bias(self):
        return torch.nn.functional.softplus(self.raw_bias)

    def forward(self, x_time, x_age, x_sex, x_smk, diag=False, **params):
        Kt = self.time(x_time, x_time, diag=diag)
        Ka = self.age(x_age,  x_age,  diag=diag)

        Ks  = self.sex(x_sex.squeeze(-1),  x_sex.squeeze(-1))
        Km  = self.smk(x_smk.squeeze(-1),  x_smk.squeeze(-1))

        Kdemo = Ka + Ks + Km

        return Kt * Kdemo 
    

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, Xt, Xage, Xsex, Xsmk, y, likelihood):
        super().__init__(train_inputs=(Xt, Xage, Xsex, Xsmk), train_targets=y, likelihood=likelihood)
        self.mean_module = ConstantMean()
        self.covar_module = MixedKernel()

    def forward(self, Xt, Xage, Xsex, Xsmk):
        mean = self.mean_module(torch.cat([Xt, Xage], dim=-1)) 
        covar = self.covar_module(Xt, Xage, Xsex, Xsmk)
        return gpytorch.distributions.MultivariateNormal(mean, covar)

likelihood = GaussianLikelihood()
model = ExactGPModel(X_time_train, X_age_train, sex_train, smk_train, y_train, likelihood)

model.train(); likelihood.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for a, b in zip(model.train_inputs, (X_time_train, X_age_train, sex_train, smk_train)):
    print("equal:", torch.equal(a, b), "shape:", a.shape, b.shape, "dtype:", a.dtype, b.dtype, "device:", a.device, b.device)

for i in range(200):  
    optimizer.zero_grad()
    output = model(*model.train_inputs)
    loss = -mll(output, y_train)
    loss.backward()
    optimizer.step()


model.eval(); likelihood.eval()

Xc_val  = torch.tensor(val_df[["Weeks_scaled","Age"]].values, dtype=torch.float32)
sex_val = torch.tensor(val_df["Sex"].map(sex_map).values, dtype=torch.long).unsqueeze(-1)
smk_val = torch.tensor(val_df["SmokingStatus"].map(smk_map).values, dtype=torch.long).unsqueeze(-1)

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    X_time_val = torch.tensor(val_df[["Weeks_scaled"]].values, dtype=torch.float32)
    X_age_val  = torch.tensor(val_df[["Age"]].values,          dtype=torch.float32)
    pred = likelihood(model(X_time_val, X_age_val, sex_val, smk_val))   
    y_mean = pred.mean.numpy()
    y_std  = pred.variance.sqrt().numpy()

equal: True shape: torch.Size([1234, 1]) torch.Size([1234, 1]) dtype: torch.float32 torch.float32 device: cpu cpu
equal: True shape: torch.Size([1234, 1]) torch.Size([1234, 1]) dtype: torch.float32 torch.float32 device: cpu cpu
equal: True shape: torch.Size([1234, 1]) torch.Size([1234, 1]) dtype: torch.int64 torch.int64 device: cpu cpu
equal: True shape: torch.Size([1234, 1]) torch.Size([1234, 1]) dtype: torch.int64 torch.int64 device: cpu cpu


RuntimeError: Boolean value of Tensor with more than one value is ambiguous